In [1]:
import numpy as np
import scipy.integrate as si
import xarray as xr
import xrcache as xc

In [2]:
# Input dataarray

x = np.arange(10)
y = np.sin(x)

# initial hash
h = xc.get_hash(np.array([x, y]))

da = xr.DataArray(y, dims="x", coords={"x": x}, name="some_array")
da.attrs.update({xc.keys.hash: h})
da

<xarray.DataArray 'some_array' (x: 10)>
array([ 0.      ,  0.841471,  0.909297,  0.14112 , -0.756802, -0.958924,
       -0.279415,  0.656987,  0.989358,  0.412118])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9
Attributes:
    hash:     040a8e4bb7f8cbfc196f77b7243a95819e09c3d2

In [3]:
# function that performs integration on the data
@xc.cached(verbose=True)
def cumtrapz(array: xr.DataArray, dx: float = 1.0, axis: int = 0):
    """perform trapz on array and return"""
    y = np.asarray(array)
    x = np.asarray(array.coords[array.dims[0]])
    i = si.cumtrapz(y, x=x, dx=dx, axis=axis, initial=0)

    # create dataarray
    da = xr.DataArray(i, dims=array.dims, coords=array.coords, attrs=array.attrs)

    return da


cumtrapz(da).attrs['xrcache_signature']

Store array to cache/some_array__cumtrapz__368.nc
Update cache/hash.json


'{"array_name": "some_array", "function_signature": {"__name__": "cumtrapz", "array": null, "dx": 1.0, "axis": 0, "__type__": "<class \'xarray.core.dataarray.DataArray\'>"}, "hash_input": "040a8e4bb7f8cbfc196f77b7243a95819e09c3d2"}'

In [4]:
cumtrapz(da)

Return stored DataArray from cache/some_array__cumtrapz__368.nc


<xarray.DataArray (x: 10)>
array([0.      , 0.420735, 1.29612 , 1.821328, 1.513487, 0.655624, 0.036454,
       0.225239, 1.048412, 1.74915 ])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9
Attributes:
    hash:               3686fb970eb610f0e9d5723fe1197a62d0b7140d
    xrcache_signature:  {"array_name": "some_array", "function_signature": {"...

In [5]:
cumtrapz(da, cache=False)

** Caching is deactivated, call function verbatim.


<xarray.DataArray (x: 10)>
array([0.      , 0.420735, 1.29612 , 1.821328, 1.513487, 0.655624, 0.036454,
       0.225239, 1.048412, 1.74915 ])
Coordinates:
  * x        (x) int64 0 1 2 3 4 5 6 7 8 9
Attributes:
    hash:     040a8e4bb7f8cbfc196f77b7243a95819e09c3d2

In [6]:
def stupid_function(array=None):
    array = array.copy()
    for i, x in enumerate(array):
        for j, y in enumerate(array):
            array[i] += i + y
    return array


@xc.cached
def cached_stupid_function(array=None):
    return stupid_function(array)

In [7]:
%%timeit
stupid_function(da)

92.9 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%%timeit
cached_stupid_function(da)

1.66 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
